In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# URL produk Tokopedia
url = "https://www.tokopedia.com/xiaomi/official-xiaomi-poco-f6-8gb-256gb-12gb-512gb-snapdragon-8s-gen-3-liquidcool-technology-4-0-iceloop-green-8-256g-fb130?extParam=cmp%3D1%26ivf%3Dfalse%26keyword%3Dhandphone&src=topads"

# Path ke ChromeDriver
driver_path = "C:/Users/DELL/Downloads/chromedriver-win64/chromedriver/chromedriver.exe"

# Buat Service ChromeDriver
service = Service(driver_path)

# Inisialisasi WebDriver
driver = webdriver.Chrome(service=service)

# Buka URL
driver.get(url)
time.sleep(5)  # Tunggu beberapa detik agar halaman termuat sepenuhnya

review_data = []
scraped_reviews = set()  # Set untuk menyimpan teks ulasan yang sudah di-scrape
max_iterations = 100  # Batas iterasi untuk mencegah loop tak berujung
iterations = 0

while iterations < max_iterations:
    iterations += 1
    
    # Klik semua tombol "Selengkapnya" untuk menampilkan ulasan lengkap
    selengkapnya_buttons = driver.find_elements(By.XPATH, "//button[@class='css-89c2tx']")
    for button in selengkapnya_buttons:
        driver.execute_script("arguments[0].click();", button)
        time.sleep(2)  # Tunggu sebentar setelah klik tombol

    # Ambil halaman sumber (HTML)
    html_source = driver.page_source

    # Parsing HTML menggunakan BeautifulSoup
    soup = BeautifulSoup(html_source, 'html.parser')

    # Temukan semua artikel yang berisi ulasan
    reviews = soup.find_all('article', class_='css-72zbc4')

    # Iterasi melalui setiap ulasan
    for review in reviews:
        # Ambil rating dari aria-label
        rating = review.find('div', {'aria-label': True})
        if rating:
            rating_value = rating['aria-label']

        # Ambil teks ulasan
        review_text = review.find('span', {'data-testid': 'lblItemUlasan'}).get_text() if review.find('span', {'data-testid': 'lblItemUlasan'}) else ''

        # Tambahkan data ke list jika ulasan belum ada di set
        if review_text not in scraped_reviews and review_text != '':
            review_data.append({
                'Rating': rating_value,
                'Review': review_text
            })
            scraped_reviews.add(review_text)  # Tambahkan teks ulasan ke set jika tidak kosong

    try:
        # Tunggu hingga tombol halaman berikutnya tersedia dan klik
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@class='css-16uzo3v-unf-pagination-item' and @aria-label='Laman berikutnya']"))
        )
        next_button.click()
        time.sleep(5)  # Tunggu beberapa detik agar halaman berikutnya termuat
    except:
        break  # Jika tidak ada tombol halaman berikutnya atau tidak bisa diklik, keluar dari loop

# Buat DataFrame
df = pd.DataFrame(review_data)

# Simpan DataFrame ke file Excel
df.to_excel('reviewsB5.xlsx', index=False)

# Tutup WebDriver
driver.quit()


CRAWLING / SCRAPPING DATA / MENDAPATKAN DATASET ULASAN PRODUK DARI TOKOPEDIA

KODE DI ATAS DIGUNAKAN UNTUK MENGAMBIL DATA ULASAN DI TOKPED

Kode di atas melakukan scraping ulasan produk dari halaman Tokopedia menggunakan Selenium dan BeautifulSoup. Pertama, kode menginisialisasi browser dengan ChromeDriver dan membuka URL produk yang diinginkan. Selanjutnya, kode menunggu halaman untuk dimuat dan mengklik semua tombol "Selengkapnya" untuk menampilkan teks ulasan lengkap. Setelah itu, kode mengumpulkan data ulasan termasuk rating dan teks ulasan menggunakan BeautifulSoup, memastikan setiap ulasan yang dikumpulkan unik. Kode kemudian mencoba mengklik tombol halaman berikutnya untuk memuat lebih banyak ulasan hingga mencapai batas iterasi. Data ulasan yang berhasil dikumpulkan disimpan dalam DataFrame dan diekspor ke file Excel. Terakhir, browser ditutup untuk mengakhiri sesi scraping.